# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import re
import nltk
import pickle
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

nltk.download(["stopwords", "punkt", "wordnet", "averaged_perceptron_tagger"])
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('disaster', engine)

X = df["message"] # , "genre" # message in original language # Do not use [[]] to access message column, return Series instead of df
Y = df.iloc[:, 4:] # select all rows, but only category columns

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # detect url
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    # normalize case
    text = text.lower()
    # remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) # this case should be handled during PoS tagging
    # strip
    text = text.strip()
    # tokenize text
    tokens = word_tokenize(text)
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

In [4]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list: # Why sentences and not just the first word?
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try: # a message might be empty
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return 1
            except:
                return 0
        return 0

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),
    
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
# pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
# Y_pred = pipeline.predict(X_test)
# print(classification_report(Y_test, Y_pred, target_names=Y_test.columns))

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),
    
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
#     print(pipeline.get_params())
    
    # specify parameters for grid search
    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#         'features__text_pipeline__vect__max_df': (0.8, 1.0),
#         'features__text_pipeline__vect__max_features': (None, 2500, 5000),
#         'features__text_pipeline__tfidf__use_idf': (True, False),
#         'features__text_pipeline__tfidf__norm': ("l1", "l2"),
#         'clf__estimator__n_estimators': [50, 100],
#         'clf__estimator__min_samples_split': [2, 3, 4],
#         'clf__estimator__criterion': ("gini", "entropy"),
    }

    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=2)
    
    return cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
def display_results(model, Y_test, Y_pred):
    labels = np.unique(Y_pred)
    accuracy = (Y_pred == Y_test).mean()
    print("Labels:", labels)
    print("Accuracy:\n", accuracy)
    print("\nBest Parameters:", model.best_params_)

In [10]:
model = build_model()
model.fit(X_train, Y_train)
print("Model trained")
Y_pred = model.predict(X_test)
print("Data predicted")
display_results(model, Y_test, Y_pred)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] features__text_pipeline__vect__ngram_range=(1, 1) ...............
[CV]  features__text_pipeline__vect__ngram_range=(1, 1), total= 1.5min
[CV] features__text_pipeline__vect__ngram_range=(1, 1) ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.9min remaining:    0.0s


[CV]  features__text_pipeline__vect__ngram_range=(1, 1), total= 1.4min
[CV] features__text_pipeline__vect__ngram_range=(1, 2) ...............
[CV]  features__text_pipeline__vect__ngram_range=(1, 2), total= 2.3min
[CV] features__text_pipeline__vect__ngram_range=(1, 2) ...............
[CV]  features__text_pipeline__vect__ngram_range=(1, 2), total= 2.3min


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  9.4min finished


Model trained
Data predicted
Labels: [0 1]
Accuracy:
 related                   0.731396
request                   0.815652
offer                     0.996156
aid_related               0.565652
medical_help              0.910824
medical_products          0.946341
search_and_rescue         0.970172
security                  0.982011
military                  0.964791
child_alone               1.000000
water                     0.927276
food                      0.878844
shelter                   0.901292
clothing                  0.984164
money                     0.974631
missing_people            0.991082
refugees                  0.966636
death                     0.951568
other_aid                 0.854397
infrastructure_related    0.932503
transport                 0.953106
buildings                 0.943419
electricity               0.979397
tools                     0.994004
hospitals                 0.989699
shops                     0.995849
aid_centers               0.987085
o

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
# Feature ideas: original message language, genre, text length

### 9. Export your model as a pickle file

In [12]:
pickle.dump(model, open("my-model.pickle", 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.